<a href="https://colab.research.google.com/github/iPrinka/MITx-Micromasters-Statistics-Data-Science/blob/main/one_league_cnn_intro_pytorch_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Datasets and Convnets in `pytorch`

- Build model on `.csv` dataset for binary classification
- Build Convolutional Network Architecture with `pytorch`
- Build in Regularization with L2 Norm and Dropout

In [ ]:
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader

import torch
from torch import optim
from torchvision.transforms import ToTensor, Resize, Compose, Normalize
import torch.nn as nn 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import SGDClassifier, LogisticRegression

In [ ]:
#mounting the drive
!ls

#### Problem: Classifying Diabetes Incidence

Using your classification knowledge, build and train at least 3 different classification models, and compare their performance.  For your third model, try to use `pytorch` to build and train a simple network.  

In [ ]:
#loading csv with pandas
df = pd.read_csv('diabetes.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.info()

#### Datasets in `pytorch` 

- `Dataset`: Retrieves items from data
- `DataLoader`: Creates batches of data using the `Dataset`

In [ ]:
#creating a dataset class
class MyDataset(Dataset):
  pass

In [ ]:
#instance of the class
dset = MyDataset(df)

In [ ]:
#DataLoader with batches
loader = DataLoader(dset, batch_size = 24)

#### Training a Model

In [ ]:
#declare gpu if available
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
device

In [ ]:
#basic linear model


In [ ]:
#move to gpu


In [ ]:
#optimizer

#loss function


In [ ]:
def training_loop(epochs, optimizer, model, loss_func, loader):
  for epoch in range(1, epochs + 1):
    loss_train = 0.0
    for x, y in loader:
      x = x.to(device)
      y = y.to(device)
      #output from model
      output = model(x)
      #loss
      loss = loss_func(output, y)
      #zero out optimizer
      optimizer.zero_grad()
      #backpropagate
      loss.backward()
      #step
      optimizer.step()
      #track loss
      loss_train += loss.item()
    if epoch == 1 or epoch % 100 == 0:
      print(f'Epoch {epoch}, Loss {loss_train}.')

In [ ]:
#train for 100 epochs
training_loop(1000, optimizer, model, loss_func, loader)

In [ ]:
#evaluating
total = 0
correct = 0
with torch.no_grad():
  for x, y in loader:
    x = x.to(device)
    y = y.to(device)
    #output of model
    out = model(x)
    #turn to predictions
    _, preds = torch.max(out, dim = 1)
    #compute number correct and total
    total += y.shape[0]
    correct += int((preds == y).sum())
    

In [ ]:
#accuracy?
correct/total

In [ ]:
#baseline? -- NULL MODEL
df['Outcome'].value_counts(normalize = True)

#### Image Example

In [ ]:
from torchvision import datasets

In [ ]:
#download cifar10 as train
cifar10 = datasets.CIFAR10('.', download = True, transform = ToTensor())

In [ ]:
#transformed dataset
transformed_cifar10 = datasets.CIFAR10('.', download = True, transform = Compose([ToTensor(), Normalize((0.4914, 0.4822, 0.4465), 
                                                                                                        (0.2470, 0.2435, 0.2616))]))

In [ ]:
#transformed validation dataset
transformed_cifar10_val = datasets.CIFAR10('.', download = True, train = False, transform = Compose([ToTensor(), Normalize((0.4914, 0.4822, 0.4465), 
                                                                                                        (0.2470, 0.2435, 0.2616))]))

In [ ]:
#binarization
labels = {0:0, 1:1}
names = ['airplane', 'car']
cifar_binary = [(im, labels[label]) for im, label in transformed_cifar10 if label in [0, 1]]
cifar_binary_val = [(im, labels[label]) for im, label in transformed_cifar10_val if label in [0, 1]]

In [ ]:
plt.imshow(cifar_binary[10][0][0, :, :], cmap = 'gray')

#### Convolutional Neural Network

![](https://upload.wikimedia.org/wikipedia/commons/thumb/6/63/Typical_cnn.png/790px-Typical_cnn.png)

#### Building the Convolutional Network

$$\text{INPUT} = (C_{in}, H_{in}, W_{in})$$

$$\text{OUTPUT} = (C_{out}, H_{out}, W_{out})$$

$$H_{out} = \left\lfloor\frac{H_{in}  + 2 \times \text{padding}[0] - \text{dilation}[0]
                    \times (\text{kernel\_size}[0] - 1) - 1}{\text{stride}[0]} + 1\right\rfloor$$

$$W_{out} = \left\lfloor\frac{W_{in}  + 2 \times \text{padding}[1] - \text{dilation}[1]
                    \times (\text{kernel\_size}[1] - 1) - 1}{\text{stride}[1]} + 1\right\rfloor$$

In [ ]:
#defining the network as a module


In [ ]:
#model, optimizer, and loss
model = CNet()
optimizer = optim.SGD(model.parameters(), lr = 0.01)
loss = nn.CrossEntropyLoss()

In [ ]:
#model to gpu
model = model.to(device)

In [ ]:
#train batches
train_loader = DataLoader(cifar_binary, batch_size = 32)

In [ ]:
#train the model
training_loop(100, optimizer, model, loss, train_loader)

In [ ]:
#validation loader
val_loader = DataLoader(cifar_binary_val, batch_size = 32, shuffle = False)

In [ ]:
#validation function
def validate(model, train_loader, val_loader):
  for name, dataset in [("Train", train_loader), ("Test", val_loader)]:
    correct = 0
    total = 0
    with torch.no_grad():
      for imgs, labels in dataset:
        imgs = imgs.to(device)
        labels = labels.to(device)
        out = model(imgs)
        _, preds = torch.max(out, dim = 1)
        total += labels.shape[0]
        correct += int((preds == labels).sum())
      print(f"{name}: Accuracy: {correct/total}")

In [ ]:
#examine the accuracy
validate(model, train_loader, val_loader)

In [ ]:
#save the model
torch.save(model.state_dict(), 'car_vs_plane.pt')

In [ ]:
#reload the model
reloaded = torch.load('car_vs_plane.pt')
reloaded

#### Problem

Load in the Digit Dataset from `pytorch`.

#### Regularization

- Weight Decay
- Dropout

In [ ]:
def training_loop_l2(epochs, optimizer, model, loss_func, loader):
  for epoch in range(1, epochs + 1):
    loss_train = 0.0
    for x, y in loader:
      x = x.to(device)
      y = y.to(device)
      outputs = model(x)
      loss = loss_func(outputs, y)
      ###lambda
      l2_lambda = 0.01
      ###compute the l2 term
      l2_norm = sum(param.pow(2).sum() for param in model.parameters())
      ###update loss
      loss = loss + l2_lambda*l2_norm
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      loss_train += loss.item()
    if epoch == 1 or epoch % 10 == 0:
      print(f'Epoch {epoch}, Loss {loss_train}.')

In [ ]:
class CNetDropout(nn.Module):
  def __init__(self, n_chans1 = 16):
    super.__init__()
    self.n_chans1 = n_chans1
    self.conv1 = nn.Conv2d(3, n_chans1, 3, padding = 1)
    #add dropout
    self.conv1_dropout = nn.Dropout(.2)
    self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, 3, padding = 1)
    #add dropout
    self.conv2_dropout = nn.Dropout(.2)
    self.lin1 = nn.Linear(8*8*n_chans1 // 2, 32)
    self.lin2 = nn.Linear(32, 2)

  def forward(self, x):
    x = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
    #use dropout
    x = self.conv1_dropout(x)
    x = F.max_pool2d(torch.tanh(self.conv2(x)), 2)
    #use dropout
    x = self.conv2_dropout(x)
    x = x.view(-1, 8*8*8*self.n_chans1 // 2)
    x = torch.tanh(self.lin1(x))
    x = self.lin2(x)
    return x 
